<a href="https://colab.research.google.com/github/rocketsvm/DFDNet/blob/master/DFDNet_DFL_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font size="+4">DFDNet for DeepFaceLab</font></b>

<b><font size="+2">Based on:</font></b>

**GitHub repository**: [DFDNet](https://github.com/csxmli2016/DFDNet)

Article: [Blind Face Restoration via Deep Multi-scale Component Dictionaries](https://arxiv.org/pdf/2008.00418.pdf)

Creator: **[csxmli2016](https://github.com/csxmli2016)**

This Colab notebook based on [notebook by tg-bomze](https://github.com/tg-bomze/DFDNet)

In [ ]:
#@title <b><font size="+3">0) Check GPU</font></b>
!nvidia-smi

In [ ]:
#@title <b><font size="+3">1) Prepare Colab machine</font></b>
!pip install dominate

from IPython.display import clear_output
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import os

warnings.filterwarnings("ignore")
%matplotlib inline

!rm -rf /content/sample_data
!git clone --branch master https://github.com/rocketsvm/DFDNet.git

os.chdir('/content/DFDNet/')
!pip install -r requirements.txt

link_prefix = 'https://github.com/rocketsvm/DFDNet/releases/download/20200827/'
wget_options = '-q --show-progress --progress=bar:noscroll --no-check-certificate -r'
!wget $wget_options $link_prefix'latest_net_G.pth' -O 'latest_net_G.pth'
!wget $wget_options $link_prefix'left_eye_32_center.npy' -O 'left_eye_32_center.npy'
!wget $wget_options $link_prefix'left_eye_64_center.npy' -O 'left_eye_64_center.npy'
!wget $wget_options $link_prefix'left_eye_128_center.npy' -O 'left_eye_128_center.npy'
!wget $wget_options $link_prefix'left_eye_256_center.npy' -O 'left_eye_256_center.npy'
!wget $wget_options $link_prefix'right_eye_32_center.npy' -O 'right_eye_32_center.npy'
!wget $wget_options $link_prefix'right_eye_64_center.npy' -O 'right_eye_64_center.npy'
!wget $wget_options $link_prefix'right_eye_128_center.npy' -O 'right_eye_128_center.npy'
!wget $wget_options $link_prefix'right_eye_256_center.npy' -O 'right_eye_256_center.npy'
!wget $wget_options $link_prefix'mouth_32_center.npy' -O 'mouth_32_center.npy'
!wget $wget_options $link_prefix'mouth_64_center.npy' -O 'mouth_64_center.npy'
!wget $wget_options $link_prefix'mouth_128_center.npy' -O 'mouth_128_center.npy'
!wget $wget_options $link_prefix'mouth_256_center.npy' -O 'mouth_256_center.npy'
!wget $wget_options $link_prefix'nose_32_center.npy' -O 'nose_32_center.npy'
!wget $wget_options $link_prefix'nose_64_center.npy' -O 'nose_64_center.npy'
!wget $wget_options $link_prefix'nose_128_center.npy' -O 'nose_128_center.npy'
!wget $wget_options $link_prefix'nose_256_center.npy' -O 'nose_256_center.npy'
!wget $wget_options $link_prefix'vgg19.pth' -O 'vgg19.pth'
!mkdir -p /content/DFDNet/checkpoints/facefh_dictionary
!mv latest_net_G.pth /content/DFDNet/checkpoints/facefh_dictionary
!mkdir -p /content/DFDNet/DictionaryCenter512
!mv *_center.npy /content/DFDNet/DictionaryCenter512
!mkdir -p /content/DFDNet/weights
!mv vgg19.pth /content/DFDNet/weights

!nvidia-smi -L

# print("0%/100%   Copying has started and will take about 4 minutes.")
# !rsync -ah --progress '/content/drive/My Drive/DFDNet/checkpoints/facefh_dictionary/latest_net_G.pth' '/content/DFDNet/checkpoints/facefh_dictionary'
# print("33%/100%  Checkpoints copied")
# !rsync -ah --progress  '/content/drive/My Drive/DFDNet/weights/vgg19.pth' '/content/DFDNet/weights/'
# print("66%/100%  Weights copied")
# !rsync -ah --progress -r '/content/drive/My Drive/DFDNet/DictionaryCenter512/' '/content/DFDNet/DictionaryCenter512/'
# print("100%/100% Dictionary copied")

In [ ]:
#@title <b><font size="+3">2) Mount Google Drive</font></b>

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#@title <b><font size="+3">3) Enchance!</font></b>

import os

working_dir = "/content/drive/My Drive/" #@param {type:"string"}
aligned_zip = False #@param {type:"boolean"}
jpeg_quality = 95 #@param {type:"slider", min:90, max:100, step:1}

#@markdown Pick a folder from your Google Drive.

#@markdown **working_dir** should contain **aligned** subfolder or **aligned.zip** file with JPG images that were extracted by DeepFaceLab *'faceset extract'* scripts.

#@markdown Results will be saved in **aligned_dfdn** subfolder or **aligned_dfdn.zip** file in **working_dir**.

#@markdown If working not in zip mode existing files will be skipped so you can easily continue in case of disconnect.

if aligned_zip:
  zip_path = os.path.join(working_dir,'aligned.zip')
  if not os.path.isfile(zip_path):
    print('"{}" not found'.format(zip_path))
    sys.exit()
  !rm -r '/content/aligned'
  !rm -r '/content/aligned_dfdn'
  !rsync -ah --progress "$zip_path" '/content/'
  !mkdir -p '/content/aligned'
  !7z e '/content/aligned.zip' -o'/content/aligned'
  !rm '/content/aligned.zip'
  w_dir = '/content/'
  tmp_dir = ''
else:
  in_path = os.path.join(working_dir,'aligned')
  out_path = os.path.join(working_dir,'aligned_dfdn')
  if not os.path.isdir(in_path):
    print('"{}" not found'.format(in_path))
    sys.exit()
  !mkdir -p "$out_path"
  w_dir = working_dir
  tmp_dir = '/content/'

os.chdir('/content/DFDNet/')
!python -W ignore test_DFL.py --working_dir "$w_dir" --jpeg_quality=$jpeg_quality --tmp_dir="$tmp_dir"

if aligned_zip:
  !rm /content/aligned_dfdn.zip
  !7z a /content/aligned_dfdn.zip /content/aligned_dfdn/*
  output_zip = os.path.join(working_dir,'aligned_dfdn.zip')
  !rsync -ah --progress /content/aligned_dfdn.zip "$output_zip"
  print('Results saved to "{}"'.format(output_zip))